In [1]:
# time library 
import datetime

# widget library
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

# DS18B20 1-Wire library (by default GPIO 4 is used with sensor (Temperature)
from w1thermsensor import W1ThermSensor 
ds18b20Sensor = W1ThermSensor() 

# Adafruit DHT library (Temperature/Humidity)
import Adafruit_DHT
DHT22Sensor = Adafruit_DHT.DHT22

# BMP library (Pressure / Temperature)
import Adafruit_BMP.BMP085 as BMP085 
bmp180Sensor = BMP085.BMP085()

# Rapberry Pi GPIOs
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

In [2]:
#define sensors GPIOs
button = 20
DHTpin = 16

#define actuators GPIOs
global ledRed, ledYlw, ledGrn
ledRed = 13
ledYlw = 19
ledGrn = 26

In [3]:

# Define button pin as an input
GPIO.setup(button, GPIO.IN)   

# Define led pins as output
GPIO.setup(ledRed, GPIO.OUT)   
GPIO.setup(ledYlw, GPIO.OUT) 
GPIO.setup(ledGrn, GPIO.OUT) 

# Turn leds OFF 
GPIO.output(ledRed, GPIO.LOW)
GPIO.output(ledYlw, GPIO.LOW)
GPIO.output(ledGrn, GPIO.LOW)

In [4]:
# Convert data in text
def convTxt(data):
	if data == 1:
		data = 'ON'
	else:
		data = 'OFF'
	return data

In [5]:
# Display on screen important data
def printData():
    print ("Local Station Time:            ", timeString)
    print ("External Temperature (Aux.):   ", tempExt, "oC")
    print ("Station Air Temperature:       ", tempLab, "oC")
    print ("Station Air Humidity:          ", humLab, "%")
    print ("Sea Level Air Pressure:        ", presSL, "mBar")
    print ("Absolute Station Air Pressure: ", presAbs, "mBar")
    print ("Station Calculated Altitude:   ", altLab, "m")

In [6]:
# Display on screen GPIO Status
def printGpioStatus():
    print ("Button Status:                 ", buttonSts)
    print ("Red Led Status:                ", ledRedSts)
    print ("Yellow Led Status:             ", ledYlwSts)
    print ("Green Led Status:              ", ledGrnSts)

In [7]:
# Read data from BMP180
def bmp180GetData(altitude):
	
	temp = bmp180Sensor.read_temperature()
	pres = bmp180Sensor.read_pressure()
	alt =  bmp180Sensor.read_altitude()
	presSeaLevel = pres / pow(1.0 - altitude/44330.0, 5.255) 
	
	temp = round (temp, 1)
	pres = round (pres/100, 2) # absolute pressure in hPa (or mbar)
	alt = round (alt)
	presSeaLevel = round (presSeaLevel/100, 2) # absolute pressure in hPa (or mbar)

	return temp, pres, alt, presSeaLevel

In [8]:
# Get data (from local sensors)
def getSensorData(altReal=0):
	global timeString
	global humLab
	global tempExt
	global tempLab
	global presSL
	global altLab
	global presAbs
	global buttonSts
	
	# Get time of reading
	now = datetime.datetime.now()
	timeString = now.strftime("%Y-%m-%d %H:%M")
	
	# Read External Temperature (1 meter distance)
	tempExt = round(ds18b20Sensor.get_temperature(), 1)
	
	tempLab, presAbs, altLab, presSL = bmp180GetData(altReal) 
	
	humDHT, tempDHT = Adafruit_DHT.read_retry(DHT22Sensor, DHTpin)
	if humDHT is not None and tempDHT is not None:
		humLab = round (humDHT)

In [9]:
# Get GPIO status data 
def getGpioStatus():
    global timeString
    global buttonSts
    global ledRedSts
    global ledYlwSts
    global ledGrnSts

    # Get time of reading
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")
    
    # Read GPIO Status
    buttonSts = convTxt(GPIO.input(button))
    ledRedSts = convTxt(GPIO.input(ledRed))
    ledYlwSts = convTxt(GPIO.input(ledYlw))
    ledGrnSts = convTxt(GPIO.input(ledGrn))

In [10]:
# Acting on GPIOs and printing Status
def controlLeds(r, y, g):
    if (r):
        GPIO.output(ledRed, GPIO.HIGH)
    else:
        GPIO.output(ledRed, GPIO.LOW)        
    if (y):
        GPIO.output(ledYlw, GPIO.HIGH)
    else:
        GPIO.output(ledYlw, GPIO.LOW)
    if (g):
        GPIO.output(ledGrn, GPIO.HIGH)
    else:
        GPIO.output(ledGrn, GPIO.LOW)
    
    getGpioStatus()
    printGpioStatus()

In [19]:
alt = 950 # real altitude of weather Station
getSensorData(alt)
printData()

Local Station Time:             2021-10-06 20:03
External Temperature (Aux.):    22.4 oC
Station Air Temperature:        25.1 oC
Station Air Humidity:           58 %
Sea Level Air Pressure:         1130.77 mBar
Absolute Station Air Pressure:  1009.1 mBar
Station Calculated Altitude:    34 m


In [17]:
f = interactive(controlLeds, r=(0,1,1), y=(0,1,1), g=(0,1,1))
display(f)

interactive(children=(IntSlider(value=0, description='r', max=1), IntSlider(value=0, description='y', max=1), …